# Examen de la práctica 2 de APR, grupo 4CO21, 10 de enero de 2024
# Turno 1: de 19h a 19.45h

## El conjunto de datos caltech-101

Caltech-101 consta de imágenes de objetos de 101 clases, más una clase "cajón de sastre". Cada imagen está etiquetada con un único objeto. Cada clase contiene entre 40 y 800 imágenes aproximadamente, con alrededor de 9.000 imágenes en total. Las imágenes son de tamaños variables, de anchos y altos típicamente entre 200 y 300 píxeles. A continuación se carga este conjunto de datos mediante la librería tensorflow_datasets y se divide en una parte para entrenamiento (train_data), otra para validación (val_data) y otra para test (test_data).

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
train_data, val_data, test_data = tfds.load('caltech101', split=['train[:80%]', 'train[80%:]', 'test'], as_supervised=True)

## Ejercicio

Como hicimos en las sesiones 3 y 4 de la práctica, haz un fine-tuning (ajuste fino) de una red Keras pre-entrenada con ImageNet que clasifique el test de caltech-101 con la máxima precisión posible, preferiblemente superior al 80%.

In [ ]:
train_size = len(train_data)
from keras.applications.inception_v3 import preprocess_input

img_size = (200, 300)
num_classes = 101

def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    label = tf.one_hot(label, num_classes)
    return image, label

train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)

train_size = int(0.8 * train_size)
train_dataset = train_data.take(train_size)
val_dataset = train_data.skip(train_size)
test_dataset = test_data

print(len(train_dataset),len(val_dataset))

from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, RandomRotation, RandomTranslation, RandomZoom
from keras.models import Model

input_layer = Input(shape=img_size + (3,))

x = RandomRotation(factor=0.1, fill_mode='nearest')(input_layer)
x = RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='nearest')(x)
x = RandomZoom(height_factor=0.2, fill_mode='nearest')(x)

inception_model = InceptionV3(input_shape=img_size + (3,),include_top=False, weights='imagenet')

for layer in inception_model.layers:
    layer.trainable = False

x = inception_model(x)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

aug_model = Model(inputs=input_layer, outputs=output)

aug_model.summary()

from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
aug_model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=10
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = aug_model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

aug_model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = aug_model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Dataset caltech101 downloaded and prepared to /root/tensorflow_datasets/caltech101/3.0.2. Subsequent calls will reuse this data.
1958 490
Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/inception_v3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
87910968/87910968 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 input_1 (InputLayer)        [(None, 200, 300, 3)]     0         
                                                                 
 random_rotation (RandomRot  (None, 200, 300, 3)       0         
 ation)                                                          
                                                                 
 random_translation (Random  (None, 200, 300, 3)       0         
 Translation)                                                    
                                                                 
 random_zoom (RandomZoom)    (None, 200, 300, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 4, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 101)               103525    
                                                                 
=================================================================
Total params: 24004485 (91.57 MB)
Trainable params: 2201701 (8.40 MB)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________
Epoch 1/10
62/62 [==============================] - ETA: 0s - loss: 3.9678 - accuracy: 0.2114
Epoch 1: val_accuracy improved from -inf to 0.67755, saving model to best_model.h5
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(

62/62 [==============================] - 33s 249ms/step - loss: 3.9678 - accuracy: 0.2114 - val_loss: 1.9891 - val_accuracy: 0.6776 - lr: 0.0010
Epoch 2/10
61/62 [============================>.] - ETA: 0s - loss: 1.9746 - accuracy: 0.5461
Epoch 2: val_accuracy improved from 0.67755 to 0.79592, saving model to best_model.h5
62/62 [==============================] - 8s 128ms/step - loss: 1.9687 - accuracy: 0.5475 - val_loss: 1.0298 - val_accuracy: 0.7959 - lr: 0.0010
Epoch 3/10
61/62 [============================>.] - ETA: 0s - loss: 1.3218 - accuracy: 0.6675
Epoch 3: val_accuracy improved from 0.79592 to 0.81224, saving model to best_model.h5
62/62 [==============================] - 9s 139ms/step - loss: 1.3184 - accuracy: 0.6685 - val_loss: 0.8154 - val_accuracy: 0.8122 - lr: 0.0010
Epoch 4/10
61/62 [============================>.] - ETA: 0s - loss: 1.1234 - accuracy: 0.7147
Epoch 4: val_accuracy did not improve from 0.81224
62/62 [==============================] - 8s 123ms/step - loss: 1.1212 - accuracy: 0.7150 - val_loss: 0.7251 - val_accuracy: 0.8102 - lr: 0.0010
Epoch 5/10
61/62 [============================>.] - ETA: 0s - loss: 1.0036 - accuracy: 0.7218
Epoch 5: val_accuracy improved from 0.81224 to 0.84082, saving model to best_model.h5
62/62 [==============================] - 9s 147ms/step - loss: 1.0006 - accuracy: 0.7227 - val_loss: 0.6300 - val_accuracy: 0.8408 - lr: 0.0010
Epoch 6/10
61/62 [============================>.] - ETA: 0s - loss: 0.8891 - accuracy: 0.7510
Epoch 6: val_accuracy did not improve from 0.84082
62/62 [==============================] - 7s 121ms/step - loss: 0.8869 - accuracy: 0.7518 - val_loss: 0.6578 - val_accuracy: 0.8163 - lr: 0.0010
Epoch 7/10
61/62 [============================>.] - ETA: 0s - loss: 0.8030 - accuracy: 0.7828
Epoch 7: val_accuracy did not improve from 0.84082
62/62 [==============================] - 8s 124ms/step - loss: 0.8011 - accuracy: 0.7829 - val_loss: 0.5789 - val_accuracy: 0.8388 - lr: 0.0010
Epoch 8/10
61/62 [============================>.] - ETA: 0s - loss: 0.7508 - accuracy: 0.7802
Epoch 8: val_accuracy did not improve from 0.84082
62/62 [==============================] - 8s 126ms/step - loss: 0.7486 - accuracy: 0.7809 - val_loss: 0.5789 - val_accuracy: 0.8408 - lr: 0.0010
Epoch 9/10
61/62 [============================>.] - ETA: 0s - loss: 0.7078 - accuracy: 0.8012
Epoch 9: val_accuracy did not improve from 0.84082
62/62 [==============================] - 8s 124ms/step - loss: 0.7058 - accuracy: 0.8018 - val_loss: 0.5676 - val_accuracy: 0.8245 - lr: 0.0010
Epoch 10/10
61/62 [============================>.] - ETA: 0s - loss: 0.6660 - accuracy: 0.7951
Epoch 10: val_accuracy improved from 0.84082 to 0.85714, saving model to best_model.h5
62/62 [==============================] - 9s 138ms/step - loss: 0.6640 - accuracy: 0.7957 - val_loss: 0.5302 - val_accuracy: 0.8571 - lr: 0.0010
Test loss: 57.08
Test accuracy: 82.77
